In [18]:
import symforce
try:
        symforce.set_epsilon_to_symbol()
except:
        pass
import symforce.symbolic as sf
import numpy as np
from symforce.notebook_util import display

In [19]:
m = sf.Matrix(np.ones((3,3)))
m

[1.0, 1.0, 1.0]
[1.0, 1.0, 1.0]
[1.0, 1.0, 1.0]

In [20]:
R = sf.Rot3.from_yaw_pitch_roll(np.pi/2,0,1)
R

<Rot3 <Q xyzw=[0.339005049421045, 0.339005049421045, 0.620544580563746, 0.620544580563746]>>

In [21]:
def symvec(shape=None):
    M = []
    # variables = []
    for i in range(shape[0]):
        M.append([])
        for j in range(shape[1]):
            M[i].append(sf.Symbol(f"x{i*shape[1]+j}"))
    return sf.Matrix(M)
M = symvec((3,1))
M.jacobian(M[0])

[1]
[0]
[0]

In [22]:
sf.V3([(sf.V3.symbolic("x") - sf.V3.symbolic("b")).norm(),
(sf.V3.symbolic("x") - sf.V3.symbolic("b")).norm(),
(sf.V3.symbolic("x") - sf.V3.symbolic("b")).norm()]).jacobian(sf.V3.symbolic("x"))

[(-b0 + x0)/sqrt((-b0 + x0)**2 + (-b1 + x1)**2 + (-b2 + x2)**2), (-b1 + x1)/sqrt((-b0 + x0)**2 + (-b1 + x1)**2 + (-b2 + x2)**2), (-b2 + x2)/sqrt((-b0 + x0)**2 + (-b1 + x1)**2 + (-b2 + x2)**2)]
[(-b0 + x0)/sqrt((-b0 + x0)**2 + (-b1 + x1)**2 + (-b2 + x2)**2), (-b1 + x1)/sqrt((-b0 + x0)**2 + (-b1 + x1)**2 + (-b2 + x2)**2), (-b2 + x2)/sqrt((-b0 + x0)**2 + (-b1 + x1)**2 + (-b2 + x2)**2)]
[(-b0 + x0)/sqrt((-b0 + x0)**2 + (-b1 + x1)**2 + (-b2 + x2)**2), (-b1 + x1)/sqrt((-b0 + x0)**2 + (-b1 + x1)**2 + (-b2 + x2)**2), (-b2 + x2)/sqrt((-b0 + x0)**2 + (-b1 + x1)**2 + (-b2 + x2)**2)]

In [23]:
# finite horizon optimization-based localization, 
# taking into account last N measurements and forming
# optimization problem to correctly assign current 
# position given N previous measurements and last estimate : )

In [24]:
ax = sf.V3.symbolic("a")
beacons = [sf.V3.symbolic(f"b{i}_") for i in range(4)]

In [28]:
J = sf.V4([(ax-beacons[i]).norm() for i in range(4)]).jacobian(ax)
display(J)

⎡                   a₀ - b₀ ₀                                        a₁ - b₀ ₁
⎢───────────────────────────────────────────────  ────────────────────────────
⎢   ____________________________________________     _________________________
⎢  ╱            2              2              2     ╱            2            
⎢╲╱  (a₀ - b₀ ₀)  + (a₁ - b₀ ₁)  + (a₂ - b₀ ₂)    ╲╱  (a₀ - b₀ ₀)  + (a₁ - b₀ 
⎢                                                                             
⎢                   a₀ - b₁ ₀                                        a₁ - b₁ ₁
⎢───────────────────────────────────────────────  ────────────────────────────
⎢   ____________________________________________     _________________________
⎢  ╱            2              2              2     ╱            2            
⎢╲╱  (a₀ - b₁ ₀)  + (a₁ - b₁ ₁)  + (a₂ - b₁ ₂)    ╲╱  (a₀ - b₁ ₀)  + (a₁ - b₁ 
⎢                                                                             
⎢                   a₀ - b₂ ₀                       

In [26]:
from typing import List
def computeH(agent_pos: sf.V3, b0: sf.V3, b1: sf.V3, b2: sf.V3, b3: sf.V3):
    J = sf.V4([
        (agent_pos-b0).norm(),
        (agent_pos-b1).norm(),
        (agent_pos-b2).norm(),
        (agent_pos-b3).norm(),
    ]).jacobian(agent_pos)
    return J

In [27]:
from symforce.codegen import Codegen, CppConfig
codegen = Codegen.function(computeH, config=CppConfig())
codegen.generate_function().generated_files[0]

PosixPath('/tmp/sf_codegen_computeH_j8m1si57/cpp/symforce/sym/computeH.h')